In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import MySQLdb

# Export Game Data

In [2]:
# Extract CSVs into DataFrames
game = "game.csv"
game_df = pd.read_csv(game)

In [3]:
game_df.head()

,game_id,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2012030221,20122013,P,2013-05-16T23:30:00Z,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
1,2012030222,20122013,P,2013-05-19T19:00:00Z,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
2,2012030223,20122013,P,2013-05-21T23:30:00Z,6,3,2,1,away win REG,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
3,2012030224,20122013,P,2013-05-23T23:00:00Z,6,3,3,4,home win OT,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
4,2012030225,20122013,P,2013-05-25T21:30:00Z,3,6,1,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT


In [4]:
team_info = "team_info.csv"
team_info_df = pd.read_csv(team_info)

# Export Team Info Data

In [5]:
team_info_df.head()

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


# Transform Game DataFrame

In [6]:
game_df['type'] = game_df['type'][game_df['type'] == 'P']
game_df = game_df.dropna()
game_df = game_df[['game_id', 'season', 'away_team_id', 'home_team_id', 'away_goals', 'home_goals','outcome']]
game_df.head()

,game_id,season,away_team_id,home_team_id,away_goals,home_goals,outcome
0,2012030221,20122013,3,6,2,3,home win OT
1,2012030222,20122013,3,6,2,5,home win REG
2,2012030223,20122013,6,3,2,1,away win REG
3,2012030224,20122013,6,3,3,4,home win OT
4,2012030225,20122013,3,6,1,3,home win REG


In [7]:
game_df= game_df.rename(columns={'away_team_id':'team_id', 'home_team_id':'home_team_id'})

game_df.head()

,game_id,season,team_id,home_team_id,away_goals,home_goals,outcome
0,2012030221,20122013,3,6,2,3,home win OT
1,2012030222,20122013,3,6,2,5,home win REG
2,2012030223,20122013,6,3,2,1,away win REG
3,2012030224,20122013,6,3,3,4,home win OT
4,2012030225,20122013,3,6,1,3,home win REG


# Transform Team_Info DataFrame

In [8]:
team_info_df = team_info_df[['team_id', 'shortName', 'teamName']]
team_info_df

,team_id,shortName,teamName
0,1,New Jersey,Devils
1,4,Philadelphia,Flyers
2,26,Los Angeles,Kings
3,14,Tampa Bay,Lightning
4,6,Boston,Bruins
5,3,NY,Rangers
6,5,Pittsburgh,Penguins
7,17,Detroit,Red Wings
8,28,San Jose,Sharks
9,18,Nashville,Predators


In [9]:
team_info_df = team_info_df.rename(columns={"team_id": "team_id" , "shortName": "CityName", "teamName": "TeamName"})
team_info_df.head()

,team_id,CityName,TeamName
0,1,New Jersey,Devils
1,4,Philadelphia,Flyers
2,26,Los Angeles,Kings
3,14,Tampa Bay,Lightning
4,6,Boston,Bruins


In [10]:
team_info_df['team_name'] = team_info_df["CityName"].str.cat(team_info_df["TeamName"], sep=' ')
team_info_df = team_info_df[['team_id','team_name']]
team_info_df = team_info_df.sort_values('team_id').reset_index()
team_info_df = team_info_df[['team_id','team_name']]
team_info_df.head()

,team_id,team_name
0,1,New Jersey Devils
1,2,NY Islanders
2,3,NY Rangers
3,4,Philadelphia Flyers
4,5,Pittsburgh Penguins


# Transform Data Into One Table

In [11]:
#game_df team_info_df away_team home_team

In [12]:
game_df = game_df.merge(team_info_df, on='team_id')

In [13]:
game_df = game_df.rename(columns={'team_name':"away_team"})
game_df = game_df.drop(columns='team_id')
game_df.head()

,game_id,season,home_team_id,away_goals,home_goals,outcome,away_team
0,2012030221,20122013,6,2,3,home win OT,NY Rangers
1,2012030222,20122013,6,2,5,home win REG,NY Rangers
2,2012030225,20122013,6,1,3,home win REG,NY Rangers
3,2012030131,20122013,15,1,3,home win REG,NY Rangers
4,2012030132,20122013,15,0,1,home win OT,NY Rangers


In [14]:
game_df = game_df.rename(columns={'home_team_id':'team_id'})
game_df = game_df.merge(team_info_df, on='team_id')
game_df.head()

,game_id,season,team_id,away_goals,home_goals,outcome,away_team,team_name
0,2012030221,20122013,6,2,3,home win OT,NY Rangers,Boston Bruins
1,2012030222,20122013,6,2,5,home win REG,NY Rangers,Boston Bruins
2,2012030225,20122013,6,1,3,home win REG,NY Rangers,Boston Bruins
3,2012030313,20122013,6,1,2,home win OT,Pittsburgh Penguins,Boston Bruins
4,2012030314,20122013,6,0,1,home win REG,Pittsburgh Penguins,Boston Bruins


In [15]:
game_df = game_df.rename(columns={'team_name':"home_team"})
game_df = game_df.drop(columns='team_id')
game_df.head()

,game_id,season,away_goals,home_goals,outcome,away_team,home_team
0,2012030221,20122013,2,3,home win OT,NY Rangers,Boston Bruins
1,2012030222,20122013,2,5,home win REG,NY Rangers,Boston Bruins
2,2012030225,20122013,1,3,home win REG,NY Rangers,Boston Bruins
3,2012030313,20122013,1,2,home win OT,Pittsburgh Penguins,Boston Bruins
4,2012030314,20122013,0,1,home win REG,Pittsburgh Penguins,Boston Bruins


In [16]:
neworder = ['game_id','season', 'away_team','home_team','away_goals','home_goals','outcome']
playoff_data = game_df.reindex(columns=neworder)
playoff_data.head()

,game_id,season,away_team,home_team,away_goals,home_goals,outcome
0,2012030221,20122013,NY Rangers,Boston Bruins,2,3,home win OT
1,2012030222,20122013,NY Rangers,Boston Bruins,2,5,home win REG
2,2012030225,20122013,NY Rangers,Boston Bruins,1,3,home win REG
3,2012030313,20122013,Pittsburgh Penguins,Boston Bruins,1,2,home win OT
4,2012030314,20122013,Pittsburgh Penguins,Boston Bruins,0,1,home win REG


In [17]:
playoff_data['winning_team'] = ''
for index, x in playoff_data.iterrows():
    if playoff_data.iloc[index,6] == 'home win OT' or playoff_data.iloc[index,6] == 'home win REG':
        playoff_data.iloc[index,7] = playoff_data.iloc[index,3]
    else:
        playoff_data.iloc[index,7] = playoff_data.iloc[index,2]
    
    
    
    
#    if x == 'home win OT':
#        playoff_data['winning_team'] = playoff_data['home_team']
#    elif x == 'away win OT':
#        playoff_data['winning_team'] = playoff_data['away_team']
            
playoff_data.head()

,game_id,season,away_team,home_team,away_goals,home_goals,outcome,winning_team
0,2012030221,20122013,NY Rangers,Boston Bruins,2,3,home win OT,Boston Bruins
1,2012030222,20122013,NY Rangers,Boston Bruins,2,5,home win REG,Boston Bruins
2,2012030225,20122013,NY Rangers,Boston Bruins,1,3,home win REG,Boston Bruins
3,2012030313,20122013,Pittsburgh Penguins,Boston Bruins,1,2,home win OT,Boston Bruins
4,2012030314,20122013,Pittsburgh Penguins,Boston Bruins,0,1,home win REG,Boston Bruins


### Load Data Into Database

In [18]:
# Create database connection
rds_connection_string = "root:Password1@localhost/playoff_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [19]:
# Confirm tables
engine.table_names()

['playoff_data']

In [20]:
# Load data into Database
playoff_data.to_sql(name='playoff_data', con=engine, if_exists='replace', index=False)

# Queries

In [21]:
# Query playoff_data table
pd.read_sql_query('select * from playoff_data', con=engine).head()

,game_id,season,away_team,home_team,away_goals,home_goals,outcome,winning_team
0,2012030221,20122013,NY Rangers,Boston Bruins,2,3,home win OT,Boston Bruins
1,2012030222,20122013,NY Rangers,Boston Bruins,2,5,home win REG,Boston Bruins
2,2012030225,20122013,NY Rangers,Boston Bruins,1,3,home win REG,Boston Bruins
3,2012030313,20122013,Pittsburgh Penguins,Boston Bruins,1,2,home win OT,Boston Bruins
4,2012030314,20122013,Pittsburgh Penguins,Boston Bruins,0,1,home win REG,Boston Bruins


In [22]:
# Query winner of NHL playoffs in 2012-2013 season
pd.read_sql_query("SELECT season, winning_team FROM playoff_data WHERE season = '20122013' GROUP BY winning_team HAVING count(winning_team) = 16", con=engine)

,season,winning_team
0,20122013,Chicago Blackhawks


In [23]:
# Query how many goals each team made in the 2012-2013 season in descending order
pd.read_sql_query("SELECT team, sum(goals_made) goals_made FROM(SELECT away_team team, sum(away_goals) goals_made FROM playoff_data WHERE season = '20122013' GROUP BY team UNION SELECT home_team team, sum(home_goals) goals_made FROM playoff_data WHERE season = '20122013' GROUP BY team) AS T GROUP BY team ORDER BY goals_made DESC", con=engine)

,team,goals_made
0,Boston Bruins,65.0
1,Chicago Blackhawks,64.0
2,Pittsburgh Penguins,49.0
3,Los Angeles Kings,37.0
4,Detroit Red Wings,33.0
5,Ottawa Senators,31.0
6,NY Rangers,26.0
7,San Jose Sharks,25.0
8,Anaheim Ducks,21.0
9,Toronto Maple Leafs,18.0


In [24]:
# Query winner of NHL playoffs in 2016-2017 season
pd.read_sql_query("SELECT season, winning_team FROM playoff_data WHERE season = '20162017' GROUP BY winning_team HAVING count(winning_team) = 16", con=engine)

,season,winning_team
0,20162017,Pittsburgh Penguins


In [25]:
# Query how many goals each team made in the 2016-2017 season in descending order
pd.read_sql_query("SELECT team, sum(goals_made) goals_made FROM(SELECT away_team team, sum(away_goals) goals_made FROM playoff_data WHERE season = '20162017' GROUP BY team UNION SELECT home_team team, sum(home_goals) goals_made FROM playoff_data WHERE season = '20162017' GROUP BY team) AS T GROUP BY team ORDER BY goals_made DESC", con=engine)

,team,goals_made
0,Pittsburgh Penguins,77.0
1,Nashville Predators,60.0
2,Anaheim Ducks,50.0
3,Ottawa Senators,47.0
4,Edmonton Oilers,36.0
5,Washington Capitals,36.0
6,NY Rangers,34.0
7,Toronto Maple Leafs,16.0
8,San Jose Sharks,14.0
9,Boston Bruins,13.0
